In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()  # this loads the .env file automatically

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"]  = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [11]:
from crewai import Agent, Task, Crew

In [12]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [13]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [14]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [15]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [16]:
from crewai_tools import BaseTool

In [17]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [18]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [21]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [22]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [23]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
)

In [24]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to start by gathering all available data on DeepLearningAI. Let's begin by listing the files in the directory to see if there are any relevant documents.

Action: List files in directory

Action Input: {}
 

File paths: 
-./instructions/enterprise_solutions_framework.md
- ./instructions/small_business_engagement.md
- ./instructions/tech_startups_outreach.md

Thought:
The listed files do not seem di

In [25]:
from IPython.display import Markdown
Markdown(result)

---

**Email Draft 1: Initial Outreach**

Subject: Empowering AI Education with CrewAI Solutions

Dear Andrew,

I hope this email finds you well.

I am writing to congratulate you on the remarkable achievements of DeepLearningAI, particularly the recent launch of innovative AI education initiatives such as "Vibe Coding 101" and the alpha testing program for the AI mentoring product. Your vision to redefine teaching through AI agents is truly inspiring and aligns perfectly with our mission at CrewAI.

At CrewAI, we are committed to providing advanced learning platforms and scalable infrastructure solutions that can enhance the educational experience for students worldwide. We believe that our cutting-edge AI technologies can complement DeepLearningAI's offerings, helping you reach a broader audience and provide personalized learning paths tailored to individual needs.

I would love the opportunity to discuss how CrewAI's solutions can support DeepLearningAI's goals and explore potential collaborative partnerships. Whether it's co-developing educational content or jointly hosting AI education events, we are eager to contribute to your mission of making world-class AI education accessible to all.

Looking forward to connecting with you soon.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI

---

**Email Draft 2: Follow-Up**

Subject: Exploring Synergies Between DeepLearningAI and CrewAI

Dear Andrew,

I hope this follow-up finds you well.

Following our initial outreach, I wanted to share some specific ways CrewAI can support DeepLearningAI's initiatives. Your recent discussions on using AI agents to redefine teaching and the launch of the AI mentoring product highlight a clear need for advanced tools and platforms.

CrewAI's advanced learning platforms integrate cutting-edge AI technologies that can enhance the learning experience for your students. Our scalable infrastructure solutions can support DeepLearningAI's rapid growth, ensuring that your educational offerings can expand seamlessly. Additionally, our personalized learning paths can be tailored to individual students' needs, making the learning experience more effective and engaging.

I would be delighted to schedule a call or a meeting to demonstrate how our solutions can benefit DeepLearningAI. We can also discuss potential collaborative partnerships, such as co-developing educational content or hosting joint AI education events.

Looking forward to your response.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI

---

**Email Draft 3: Value Demonstration**

Subject: Demonstrating CrewAI's Value for DeepLearningAI

Dear Andrew,

I hope this email finds you well.

Following our previous communications, I wanted to provide a more detailed overview of how CrewAI's solutions can align with DeepLearningAI's goals. Your recent product launches and public discussions underscore the importance of innovative AI education tools.

CrewAI's advanced learning platforms can integrate seamlessly with your existing offerings, providing students with an enriched learning experience. Our scalable infrastructure solutions can support your growing student base, ensuring that your educational programs can scale efficiently. Additionally, our personalized learning paths can be customized to meet the unique needs of each student, enhancing the overall effectiveness of your educational initiatives.

I would be happy to arrange a demo or provide case studies that showcase the impact of our solutions. We are confident that CrewAI can be a valuable partner in achieving DeepLearningAI's mission of making world-class AI education accessible to all.

Looking forward to the possibility of working together.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI

---